In [1]:
import pandas as pd
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors
from gensim import downloader
import gzip 
import math
import itertools
from time import time
from tqdm import tqdm
import tqdm.notebook as tq
import os
os.getcwd()

'G:\\My Drive\\KU\\Thesis'

**Format for ngram documents:**

ngram TAB year TAB match_count TAB page_count TAB volume_count NEWLINE

In [2]:

import psutil
psutil.virtual_memory()

svmem(total=25617727488, available=9309523968, percent=63.7, used=16308203520, free=9309523968)

## Parallel with mgzip

# Partitioning and deleting entries under 40 matches

In [3]:
import mgzip
import string
from datetime import datetime
from time import time
import shutil
import logging

#Differences with this one: it adds up the match count of all the years in a decade and writes one line per decade, 
# threshold is reduced to 25
# gets rid of lines with punctuation

class ParrNgramFiler():
    def __init__(self, coll, out, dire, start_yr, end_yr, matchmin=25, test=False, log=False, print=False):
        self.coll = coll
        self.out = out
        self.dir = dire
        self.start_yr = start_yr
        self.end_yr = end_yr
        self.decades = [start_yr+i*10 for i in range((end_yr-start_yr)//10)]
        self.matchmin = matchmin
        self.exclist = string.punctuation + string.digits
        self.punc = string.punctuation
        self.lines = {}
        self.test = test
        self.log = log
        self.print = print
    
    def write(self, ngram, match_count, current_dec):
        split = []
        for word in ngram.lower().split():
            word = word.split("_")[0]
            if word in self.punc:
                continue
            else:
                split.append(word)
        if len(split) < 5:
            pass
        else:
            to_write = " ".join(split + ["\t" + str(match_count) + "\n"]) #Get rid of POS tagging on end of words
            self.write_dic[current_dec].write(to_write)
            #if self.log != False and current_dec == 1900:
                 #print("\n\n***************************debugg***************************\n", to_write, str(current_dec), "\n")
               # logging.debug(f"\n\n***************************debugg***************************\nto write:{to_write}  dec:{current_dec}\n")
            split = " ".join(split)
           # try: 
           #     self.ngram_dic[current_dec][split]
           #     print("DOUBLE!!!! ", split, ", ", current_dec)
           # except:
           #     print("Written: ", split, ", ", current_dec)
           #     self.write_dic[current_dec].write(to_write)
           #     self.ngram_dic[current_dec][split] = True
        
    def file(self): 
        os.chdir(f"{self.dir}/raw_data/{self.coll}")
        self.write_dic = {}
        self.size_dic = {}
        output_f = f"{self.dir}/processed_data/{self.coll}_{self.out}/"
        #self.ngram_dic = {}
        
        if self.test:
            shutil.rmtree(output_f)
        os.makedirs(output_f)
        
        for dec in self.decades:
            fname = str(dec) + ".gz"
            self.write_dic[dec] = mgzip.open(output_f + fname, "wt", encoding="utf-8")
            #self.ngram_dic[dec] = {}
        
        if self.log == True:
            logging.basicConfig(filename=f'{output_f}log_{fname}.log', encoding='utf-8', level=logging.DEBUG)
        
        #Try on specified no. files if testing
        if self.test != False:
            file_list = os.listdir()[:self.test]
        else:
            file_list = os.listdir()
        
        
        for filname in tq.tqdm(file_list):
            
            #Skip over the POS tag files
            if filname[0] == "_":
                continue
            start = time()
            print(f"Time taken on previous iter: ", time()-start)
            print("\nFile ", filname, " starting...")
            #Open file with parallel gzip 
            with mgzip.open(f"{self.dir}/raw_data/{self.coll}/" + filname, "rt", encoding="utf-8") as fil:
                ngrams = {dec:{} for dec in self.decades}
                #Try to counteract "not a gzipped file" error
                self.lines[filname] = 0
                try:
                    for i, line in enumerate(tq.tqdm(fil)):

                        #count lines for printing after each file
                        self.lines[filname] += 1

                        #Split file at the tab
                        line = gensim.utils.to_unicode(line).split("\t")

                        #Get the year as integer, throw up an error if no year fuond
                        try:
                            year = int(line[1])
                        except ValueError:
                            print(f"Year error on line: {i} -> {line}")
                            continue

                        #Get corresponding decade
                        dec = int(str(year)[:-1] + "0")

                        #Isolate ngram
                        ngram = line[0]

                        #Get match count
                        try: 
                            matches = int(line[2])
                        except Exception as e:
                            print(f"Match error on line {i} -> {line}:\n", e)
                            continue

                        #Initialise values for decadising ngram data
                        if i == 0:
                            prev_ngram = ngram
                            match_count = matches
                            current_dec = dec
                            continue

                        #Print first 1000 if in test mode
                       # if self.print != False:
                        #    if i < self.print:
                         #       if  dec == 1900:
                          #          #print("\nCurrent ngram: ", prev_ngram, "\nMatches: ", match_count, "\n Current Line: ", line)
                           #         logging.debug(f"\nCurrent ngram: {prev_ngram}\nMatches: {match_count}\nCurrent Line: {line}")
                            #else:
                             #   break

                        #if self.log == True:
                         #   logging.debug("\nCurrent ngram: ", prev_ngram, "\nMatches: ", match_count, "\n Current Line: ", line)

                        #check current/previous ngram
                        if prev_ngram == ngram:
                            #check current/previous decade
                            if dec == current_dec:
                                match_count += matches
                            #if not equal, is match coutn above?
                            else: 
                                if match_count > self.matchmin:
                                    self.write(ngram, match_count, current_dec)
                                current_dec = dec
                                match_count = matches
                        else:
                            if match_count > self.matchmin:
                                self.write(ngram, match_count, current_dec)
                            prev_ngram = ngram
                            match_count = matches
                            current_dec = dec
             #Error opening file     
                except Exception as e:
                    print("Error", e)
        #Close files once done
        for dec in self.decades:
            self.write_dic[dec].close()
                    




In [4]:
import mgzip
import string
from datetime import datetime
from time import time
import shutil
import logging

#Differences with this one: it adds up the match count of all the years in a decade and writes one line per decade, 
# threshold is reduced to 25
# gets rid of lines with punctuation

class FullNgramFiler():
    def __init__(self, coll, out, dire, start_yr, end_yr, matchmin=25, test=False, log=False, print=False):
        self.coll = coll
        self.out = out
        self.dir = dire
        self.start_yr = start_yr
        self.end_yr = end_yr
        self.decades = [start_yr+i*10 for i in range((end_yr-start_yr)//10)]
        self.matchmin = matchmin
        self.exclist = string.punctuation + string.digits
        self.punc = string.punctuation
        self.lines = {}
        self.test = test
        self.log = log
        self.print = print
    
    def write(self, ngram, match_count, dec):
        
        to_write = ngram + "\t" + str(match_count) + "\n" #Get rid of POS tagging on end of words
        self.write_dic[dec].write(to_write)
        
    def file(self): 
        os.chdir(f"{self.dir}/raw_data/{self.coll}")
        self.write_dic = {}
        self.size_dic = {}
        output_f = f"{self.dir}/processed_data/{self.coll}_{self.out}/"
        
        if self.test:
            shutil.rmtree(output_f)
        #os.makedirs(output_f)
        
        for dec in self.decades:
            fname = str(dec) + ".gz"
            self.write_dic[dec] = gzip.open(output_f + fname, "at", encoding="utf-8")
            #self.ngram_dic[dec] = {}
        
        #if self.log == True:
         #   logging.basicConfig(filename=f'{output_f}log_{fname}.log', encoding='utf-8', level=logging.DEBUG)
        
        #Try on specified no. files if testing
        if self.test != False:
            file_list = os.listdir()[:self.test]
        else:
            file_list = os.listdir()
        
        
        for filname in tq.tqdm(file_list):
            
            #Skip over the POS tag files
            if filname[0] == "_":
                continue
            start = time()
            print(f"Time taken on previous iter: ", time()-start)
            print("\nFile ", filname, " starting...")
            #Open file with parallel gzip 
            with gzip.open(f"{self.dir}/raw_data/{self.coll}/" + filname, "rt", encoding="utf-8") as fil:
                ngrams = {dec:{} for dec in self.decades}
                #Try to counteract "not a gzipped file" error
                self.lines[filname] = 0
                try:
                    for i, line in enumerate(tq.tqdm(fil)):
                        if i < 100:
                            continue
                        #count lines for printing after each file
                        self.lines[filname] += 1

                        #Split file at the tab
                        line = gensim.utils.to_unicode(line).split("\t")

                        #Get the year as integer, throw up an error if no year fuond
                        try:
                            year = int(line[1])
                        except ValueError:
                            print(f"Year error on line: {i} -> {line}")
                            continue

                        #Get corresponding decade
                        dec = int(str(year)[:-1] + "0")

                        #Isolate ngram
                        ngram = line[0].lower()
                        split = []
                        for word in ngram.lower().split():
                            word = word.split("_")[0]
                            if word in self.punc:
                                continue
                            else:
                                split.append(word)
                        if len(split) < 5:
                            continue
                        ngram = " ".join(split)

                        #Get match count
                        try: 
                            matches = int(line[2])
                        except Exception as e:
                            print(f"Match error on line {i} -> {line}:\n", e)
                            continue
                        self.write(ngram, matches, dec)
                        #ngrams[dec][ngram] += matches
                except Exception as e:
                    print(e)

                
            #for dec in self.decades:
             #   for ngram in ngrams[dec].keys(): 
              #      if  ngrams[dec][ngram] > 25:
               #         self.write(ngram, ngrams[dec][ngram], dec)
                        
        #Close files once done
        for dec in self.decades:
            self.write_dic[dec].close()
                    

In [5]:
start = time()
Filer = FullNgramFiler(coll="us_12", out="final_burst", dire="D:/google_ngrams", log=False, test=False, print=500000000000, start_yr=1500, end_yr=2010, matchmin=25)
Filer.file()
print(f"Total time taken: {time()-start}")

  0%|          | 0/729 [00:00<?, ?it/s]

Time taken on previous iter:  0.0

File  a_  starting...


0it [00:00, ?it/s]

Time taken on previous iter:  0.0

File  aa  starting...


0it [00:00, ?it/s]

...

Not a gzipped file (b'\x80\x04')
Time taken on previous iter:  0.0

File  w2vmodel_ng5_cz_5  starting...


0it [00:00, ?it/s]

Not a gzipped file (b'\x80\x04')
Time taken on previous iter:  0.0

File  w2vmodel_ng5_cz_vocab  starting...


0it [00:00, ?it/s]

Not a gzipped file (b'\x80\x04')
Total time taken: 348162.5603232384


## Check whether each file iterated over writes more information to the output files

In [6]:
ParrNgramFiler.size_dic



AttributeError: type object 'ParrNgramFiler' has no attribute 'size_dic'

In [ ]:
os.listdir()[185]

In [ ]:
list = [i for i in range(10)]
dic = {i: {} for i in range(10)}
dic

In [ ]:
a += 1